In [1]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np
import nibabel as nib
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from scipy import stats
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import pandas as pd
%matplotlib inline 
%autosave 30
sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})
sns.set(palette="colorblind")
import nitime
import re
from collections import Counter
from statistics import mean
import operator

# Import the time-series objects:
from nitime.timeseries import TimeSeries

# Import the analysis objects:
from nitime.analysis import SpectralAnalyzer, FilterAnalyzer, NormalizationAnalyzer
!ulimit -n 10000

Autosaving every 30 seconds


In [2]:
typ = {'ScanDir ID': str}
p_data = pd.read_csv("./phenotypic_info/NYU_phenotypic.csv",usecols=["ScanDir ID", "Site", "Gender","Age","DX"],dtype=typ)

df1 = pd.read_csv("./phenotypic_info/Peking_1_phenotypic.csv",usecols=["ScanDir ID", "Site", "Gender","Age","DX"],dtype=typ)
p_data = p_data.append(df1)

df2 = pd.read_csv("./phenotypic_info/Peking_2_phenotypic.csv",usecols=["ScanDir ID", "Site", "Gender","Age","DX"],dtype=typ)
p_data = p_data.append(df2)

df3 = pd.read_csv("./phenotypic_info/Peking_3_phenotypic.csv",usecols=["ScanDir ID", "Site", "Gender","Age","DX"],dtype=typ)
p_data = p_data.append(df3)

df4 = pd.read_csv("./phenotypic_info/NeuroIMAGE_phenotypic.csv",usecols=["ScanDir ID", "Site", "Gender","Age","DX"],dtype=typ)
p_data = p_data.append(df4)

p_data

,ScanDir ID,Site,Gender,Age,DX
0,1000804,5,1.0,7.29,0
1,1023964,5,1.0,8.29,3
2,1057962,5,1.0,8.78,1
3,1099481,5,0.0,8.04,1
4,1127915,5,0.0,12.44,0
...,...,...,...,...,...
43,7504392,4,0.0,18.01,1
44,8387093,4,1.0,18.59,2
45,8409791,4,1.0,17.59,0
46,8991934,4,0.0,17.01,0


In [3]:
p_data.groupby(['Site','DX']).count()

ScanDir ID  Gender  Age
Site DX                         
1    0          116     116  116
     1           29      29   29
     3           49      49   49
4    0           23      23   23
     1           18      18   18
     2            6       6    6
     3            1       1    1
5    0           99      99   99
     1           77      76   77
     2            2       2    2
     3           44      44   44

In [4]:
p_data['ScanDir ID'].duplicated().any()

False

In [5]:
p_data['ScanDir ID'].where(p_data['ScanDir ID']== '10001').any()

'10001'

In [6]:
for entry in p_data.iterrows():
    if entry[1]['ScanDir ID'] == '10001':
        print(entry[1])

ScanDir ID    10001
Site              5
Gender            0
Age           11.17
DX                3
Name: 93, dtype: object


In [7]:
p_data.dtypes

ScanDir ID     object
Site            int64
Gender        float64
Age           float64
DX              int64
dtype: object

In [8]:
files = os.listdir("./nd/")
print(len(files))

638


In [9]:
scans = {}
for filename in os.listdir("./nd/"):
    print(filename);
    s = nib.load("./nd/"+filename)
    m = re.match("^[a-z]*([0-9]*).*$",filename)
    t = s.shape[3]
    if t > 172:
        s = s.slicer[...,(t-172):]
    scans[m.groups()[0]] = s
scans

sfnwmrda0010001_session_1_rest_1.nii
sfnwmrda1023964_session_1_rest_1.nii
sfnwmrda1023964_session_1_rest_2.nii
sfnwmrda1127915_session_1_rest_1.nii
sfnwmrda1127915_session_1_rest_2.nii
sfnwmrda1208795_session_1_rest_1.nii
sfnwmrda1208795_session_1_rest_2.nii
sfnwmrda1320247_session_1_rest_1.nii
sfnwmrda1320247_session_1_rest_2.nii
sfnwmrda1411536_session_1_rest_1.nii
sfnwmrda1435954_session_1_rest_1.nii
sfnwmrda1469171_session_1_rest_1.nii
sfnwmrda1471736_session_1_rest_1.nii
sfnwmrda1561488_session_1_rest_1.nii
sfnwmrda1562298_session_1_rest_1.nii
sfnwmrda1588809_session_1_rest_1.nii
sfnwmrda1628610_session_1_rest_1.nii
sfnwmrda1643780_session_1_rest_1.nii
sfnwmrda1662160_session_1_rest_1.nii
sfnwmrda2030383_session_1_rest_1.nii
sfnwmrda2030383_session_1_rest_2.nii
sfnwmrda2081754_session_1_rest_1.nii
sfnwmrda2106109_session_1_rest_1.nii
sfnwmrda2367157_session_1_rest_1.nii
sfnwmrda2377207_session_1_rest_1.nii
sfnwmrda2427408_session_1_rest_1.nii
sfnwmrda2493190_session_1_rest_1.nii
s

sfnwmrda4116166_session_1_rest_1.nii
sfnwmrda4116166_session_1_rest_2.nii
sfnwmrda4265987_session_1_rest_1.nii
sfnwmrda4285031_session_1_rest_1.nii
sfnwmrda4475709_session_1_rest_1.nii
sfnwmrda4562206_session_1_rest_1.nii
sfnwmrda4827048_session_1_rest_1.nii
sfnwmrda4919979_session_1_rest_1.nii
sfnwmrda5164727_session_1_rest_2.nii
sfnwmrda5193577_session_1_rest_1.nii
sfnwmrda5575344_session_1_rest_1.nii
sfnwmrda5600820_session_1_rest_1.nii
sfnwmrda5669389_session_1_rest_1.nii
sfnwmrda5971050_session_1_rest_1.nii
sfnwmrda5971050_session_1_rest_2.nii
sfnwmrda5993008_session_1_rest_1.nii
sfnwmrda6115230_session_1_rest_1.nii
sfnwmrda6206397_session_1_rest_2.nii
sfnwmrda6383713_session_1_rest_1.nii
sfnwmrda6477085_session_1_rest_1.nii
sfnwmrda7135128_session_1_rest_1.nii
sfnwmrda7253183_session_1_rest_1.nii
sfnwmrda7339173_session_1_rest_1.nii
sfnwmrda7446626_session_1_rest_1.nii
sfnwmrda8009688_session_1_rest_2.nii
sfnwmrda8191384_session_1_rest_1.nii
sfnwmrda8387093_session_1_rest_1.nii
s

sfnwmrda0010026_session_1_rest_2.nii
sfnwmrda0010028_session_1_rest_1.nii
sfnwmrda0010028_session_1_rest_2.nii
sfnwmrda0010029_session_1_rest_1.nii
sfnwmrda0010029_session_1_rest_2.nii
sfnwmrda0010030_session_1_rest_1.nii
sfnwmrda0010030_session_1_rest_2.nii
sfnwmrda0010031_session_1_rest_1.nii
sfnwmrda0010032_session_1_rest_1.nii
sfnwmrda0010033_session_1_rest_1.nii
sfnwmrda0010033_session_1_rest_2.nii
sfnwmrda0010034_session_1_rest_1.nii
sfnwmrda0010034_session_1_rest_2.nii
sfnwmrda0010035_session_1_rest_1.nii
sfnwmrda0010035_session_1_rest_2.nii
sfnwmrda0010036_session_1_rest_1.nii
sfnwmrda0010037_session_1_rest_1.nii
sfnwmrda0010037_session_1_rest_2.nii
sfnwmrda0010038_session_1_rest_1.nii
sfnwmrda0010038_session_1_rest_2.nii
sfnwmrda0010039_session_1_rest_1.nii
sfnwmrda0010039_session_1_rest_2.nii
sfnwmrda0010040_session_1_rest_1.nii
sfnwmrda0010040_session_1_rest_2.nii
sfnwmrda0010041_session_1_rest_1.nii
sfnwmrda0010042_session_1_rest_1.nii
sfnwmrda0010043_session_1_rest_1.nii
s

{'0010001': <nibabel.nifti1.Nifti1Image at 0x207f3a1a50>,
 '1023964': <nibabel.nifti1.Nifti1Image at 0x1074dd510>,
 '1127915': <nibabel.nifti1.Nifti1Image at 0x1072cfa50>,
 '1208795': <nibabel.nifti1.Nifti1Image at 0x1072cf250>,
 '1320247': <nibabel.nifti1.Nifti1Image at 0x1c1bc0fe90>,
 '1411536': <nibabel.nifti1.Nifti1Image at 0x1072b8d10>,
 '1435954': <nibabel.nifti1.Nifti1Image at 0x1d90bbd090>,
 '1469171': <nibabel.nifti1.Nifti1Image at 0x1074acf50>,
 '1471736': <nibabel.nifti1.Nifti1Image at 0x1d90bbd590>,
 '1561488': <nibabel.nifti1.Nifti1Image at 0x1067dc610>,
 '1562298': <nibabel.nifti1.Nifti1Image at 0x1067dc9d0>,
 '1588809': <nibabel.nifti1.Nifti1Image at 0x1067dcc90>,
 '1628610': <nibabel.nifti1.Nifti1Image at 0x1067dc8d0>,
 '1643780': <nibabel.nifti1.Nifti1Image at 0x1067dcf10>,
 '1662160': <nibabel.nifti1.Nifti1Image at 0x1067dcd90>,
 '2030383': <nibabel.nifti1.Nifti1Image at 0x1067f9410>,
 '2081754': <nibabel.nifti1.Nifti1Image at 0x1067dc550>,
 '2106109': <nibabel.nifti1

In [10]:
len(scans)

458

In [11]:
p_data = p_data.rename(columns={'ScanDir ID':'ID'})

In [12]:
p_data

,ID,Site,Gender,Age,DX
0,1000804,5,1.0,7.29,0
1,1023964,5,1.0,8.29,3
2,1057962,5,1.0,8.78,1
3,1099481,5,0.0,8.04,1
4,1127915,5,0.0,12.44,0
...,...,...,...,...,...
43,7504392,4,0.0,18.01,1
44,8387093,4,1.0,18.59,2
45,8409791,4,1.0,17.59,0
46,8991934,4,0.0,17.01,0


In [13]:
for entry in p_data.iterrows():
    if len(entry[1]['ID']) < 7: 
        p_data.loc[p_data['ID'] == entry[1]['ID'], 'ID'] = '00'+entry[1]['ID']

In [14]:
for entry in p_data.iterrows():
    if len(entry[1]['ID']) < 7: 
        print('hi')
    if entry[1]['ID'] == '0010001':
        print(entry[1])

ID        0010001
Site            5
Gender          0
Age         11.17
DX              3
Name: 93, dtype: object


In [15]:
#scan_data = []
#for entry in p_data.iterrows():
#    try:
#        scan_data.append(scans[entry[1]['ID']].get_data())
#    except: 
#        print(entry[1]['ID'])

In [16]:
p_data.drop( p_data[ p_data['ID'] == '0010016'].index , inplace=True)
p_data.drop( p_data[ p_data['ID'] == '0010055'].index , inplace=True)
p_data.drop( p_data[ p_data['ID'] == '0010098'].index , inplace=True)
p_data.drop( p_data[ p_data['ID'] == '0010105'].index , inplace=True)
p_data.drop( p_data[ p_data['ID'] == '0010027'].index , inplace=True)
p_data.drop( p_data[ p_data['ID'] == '0010127'].index , inplace=True)

In [17]:
len(scans)

458

In [18]:
len(p_data) 

458

In [19]:
scans_ordered = []
for entry in p_data.iterrows():
    scans_ordered.append(scans[entry[1]['ID']])

In [20]:
len(scans_ordered)
#scans_ordered contains the fmri scans in the same order as in the p_data (ID)
#p_data contains info from csv
#scans contains map of scan id to fmri scan

458

In [21]:
p_data['images'] = scans_ordered
scans_ordered = 0
scans = 0

In [22]:
p_data

,ID,Site,Gender,Age,DX,images
0,1000804,5,1.0,7.29,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...
1,1023964,5,1.0,8.29,3,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...
2,1057962,5,1.0,8.78,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...
3,1099481,5,0.0,8.04,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...
4,1127915,5,0.0,12.44,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...
...,...,...,...,...,...,...
43,7504392,4,0.0,18.01,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...
44,8387093,4,1.0,18.59,2,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...
45,8409791,4,1.0,17.59,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...
46,8991934,4,0.0,17.01,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...


In [ ]:
p_data.to_pickle("./p_data.pkl")

In [23]:
def voxel_averages (x,y,z):
    temp_data = p_data
    vox_values = []
    for entry in p_data.iterrows():
        vox_values.append(entry[1]['images'].get_data()[x,y,z,:])
    temp_data['arrays'] = vox_values
    

In [25]:
def graphIt(x,y,z): 
    positive = [0 for row in range(172)]
    negative = [0 for row in range(172)]
    print("point 1")
    for entry in p_data.iterrows():
        if entry[1]['DX'] == 0: 
            negative.append(entry[1]['images'].get_data()[x,y,z,:])
        else:
            positive.append(entry[1]['images'].get_data()[x,y,z,:])
    print("point 2")
    neg = list(map(mean, zip(*negative)))
    print("point 3")
    pos = list(map(mean, zip(*positive)))
    print("point 4")
    tot = list(map(operator.sub,neg,pos))
    #tot = list(map(operator.sub,map(mean, zip(*negative)),map(mean, zip(*positive))))
    print("point 5")
    s = sum(tot)
    print("point 6")
    #print("done " + str(x) +" "+ str(y) + " "+ str(z) + " error: " + str(s))
    #f, ax = plt.subplots(1,1, figsize=(14,5))
    #ax.plot(pos)
    #ax.plot(neg)
    return s
    


In [ ]:
print(graphIt(21,20,20))


point 1


{(0, 0, 0): 3.0516677649211488,
 (0, 0, 1): 3.8892703638739476,
 (1, 0, 0): 3.0708821772859665,
 (1, 0, 1): 3.7906466607237235,
 (2, 0, 0): 3.3327248247187526,
 (2, 0, 1): 3.6006748127329047,
 (3, 0, 0): 4.205325123824878,
 (3, 0, 1): 4.021314281999366,
 (4, 0, 0): 5.794836503686383,
 (4, 0, 1): 5.3388720262883,
 (5, 0, 0): 6.064855656790314,
 (5, 0, 1): 5.830718325293674,
 (6, 0, 0): 6.193693879482453,
 (6, 0, 1): 6.075561368561466,
 (7, 0, 0): 8.203477475588443,
 (7, 0, 1): 7.184241284820018,
 (8, 0, 0): 11.683811012859223,
 (8, 0, 1): 9.088221359299496,
 (9, 0, 0): 14.6317355397332,
 (9, 0, 1): 10.055964814382605,
 (10, 0, 0): 15.612252386985347,
 (10, 0, 1): 10.504705515108071,
 (11, 0, 0): 15.502763686701655,
 (11, 0, 1): 10.290515338652767,
 (12, 0, 0): 15.672063932026504,
 (12, 0, 1): 10.647056820889702,
 (13, 0, 0): 15.546071784832748,
 (13, 0, 1): 10.735233279396198,
 (14, 0, 0): 17.28455048071919,
 (14, 0, 1): 12.16433557390701,
 (15, 0, 0): 17.839713583991397,
 (15, 0, 1): 1

In [85]:
49,58,47

(49, 58, 47)